In [2]:
# !pip install backtrader

In [5]:
import numpy as np
import pandas as pd
from backtesting  import Backtest, Strategy
from backtesting.lib import crossover
from kite_trade import *
import ta
import datetime

# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

/Users/akash_verma/miniconda3/envs/epat/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [6]:
enctoken = "rDQaexPLUfV5LCKvFn6ccZm8/8VbiOz26Y7C6Ugnv8nUG8dkOQhEJL8qF50Oe7DAA+bPe0sopmd67fHOrrw5Nmvp1L9FAXxJcY/a4BeC1bpyBXKsYNiiA=="
kite = KiteApp(enctoken=enctoken)


In [7]:
def get_diffrent_time_data(instrument_token,start_dt, end_dt,interval):
    data = pd.DataFrame()
#     range_of_date = pd.date_range(start_dt,end_dt)

#     index_val_1 = [i for i in pd.Series(range_of_date).index if i%2 == 0]
#     index_val_2 = [i for i in pd.Series(range_of_date).index if i%2 != 0]
#     x = pd.Series(range_of_date).iloc[index_val_1].reset_index(drop = True)
#     y = pd.Series(range_of_date).iloc[index_val_2].reset_index(drop = True)
#     df_1 = pd.concat([x,y],axis = 1).rename(columns = {0:"start_dt",1:"end_dt"})
#     df_1.start_dt = pd.to_datetime(df_1.start_dt)
#     df_1.end_dt = pd.to_datetime(df_1.end_dt)
#     df_1.dropna(inplace = True)
    
#     for r,c in df_1.iterrows():
#         print(c.start_dt,c.end_dt)
#     temp = pd.DataFrame(kite.historical_data(instrument_token,c.start_dt,c.end_dt,interval))
    temp = pd.DataFrame(kite.historical_data(instrument_token,start_dt,end_dt,interval))
    data = data.append(temp)
    return data

In [8]:
# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(segment == "NSE") & (name != "")')

In [9]:
inst_filter[inst_filter.tradingsymbol.str.contains("RELI")]

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
4172,738561,2885,RELIANCE,RELIANCE INDUSTRIES,0.0,None,0.0,0.05,1,EQ,NSE,NSE
4173,3857409,15068,RELIGARE,RELIGARE ENTER..,0.0,None,0.0,0.05,1,EQ,NSE,NSE
4174,1226497,4791,RELINFRA-BE,RELIANCE INFRASTRUCTU,0.0,None,0.0,0.05,1,EQ,NSE,NSE


### approach 1

In [10]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.order = None        
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0].close, period=200)
        self.ball = bt.ind.BollingerBands(period = 20, devfactor = 2.5)
        self.rsi = bt.ind.RSI(period = 2, safediv = True)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            if self.data.close > self.sma and self.data.close < self.ball.lines.bot:
#                 # BUY, BUY, BUY!!! (with default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(exectype=bt.Order.Limit, price = .97 * self.dataclose[0])
            
        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 10):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [11]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
#     modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
#     datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

    # Create a Data Feed
    instrument_token = 884737
    start_dt = datetime.datetime(2021,1,1)    # From last & days
    end_dt = datetime.datetime.now()
    interval = "day"
    
    data = get_diffrent_time_data(instrument_token,start_dt, end_dt,interval)
#     data.date = pd.to_datetime(data.date,unit = "ms")
    data.date = pd.to_datetime(pd.to_datetime(data.date).dt.date)
    data.rename(columns = {'date': "Date"},inplace = True)
    data.set_index('Date', inplace = True)
    data.columns = ['Open','High','Low',"Close","Volumne"]
    data1 = data.copy()
    # Add the Data Feed to Cerebro
    data = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(500.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('run')
    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

TypeError: 'NoneType' object is not subscriptable

In [12]:
# cerebro.plot(volumne = True)

### approach 2

In [13]:
data

NameError: name 'data' is not defined

In [14]:
def applyindicator(df):
    df['sma200'] = df.Close.rolling(200).mean()
    df['sma20'] = df.Close.rolling(20).mean()
    df["stddev"] = df.Close.rolling(20).std()
    df['Upper'] = df['sma20'] + 2.5 * df["stddev"]
    df['Lower'] = df['sma20'] - 2.5 * df["stddev"]
    df["rsi"] = ta.momentum.rsi(df.Close,2)
    
    
def conditions(df):
    df['Buy'] = np.where((df.Close > df.sma200)
                         & (df.Close <df.Lower)
                        & (.97 * df.Close >= df.Low.shift(-1)),1,0)
    df['Sell'] = np.where(df.rsi > 50, 1,0)
    df['Buyprice'] = .97 * df.Close
    df['Sellprice'] = df.Open.shift(-1)
    
    
def matchtrades(df):
    buy_sells = df[(df.Buy == 1) | (df.Sell == 1)]
    matched_buy_sells = buy_sells[(buy_sells.Buy.diff() == 1) | (buy_sells.Sell.diff() == 1)]
    return matched_buy_sells
    
    
    
    
    

In [15]:
inst_filter

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
98,4331777,16921,20MICRONS,20 MICRONS,0.0,None,0.0,0.05,1,EQ,NSE,NSE
99,1025,4,21STCENMGM,21ST CENTURY MGMT SERVICE,0.0,None,0.0,0.05,1,EQ,NSE,NSE
152,1595393,6232,3IINFOLTD,3I INFOTECH,0.0,None,0.0,0.05,1,EQ,NSE,NSE
153,121345,474,3MINDIA,3M INDIA,0.0,None,0.0,0.05,1,EQ,NSE,NSE
154,664321,2595,3PLAND,3P LAND HOLDINGS,0.0,None,0.0,0.05,1,EQ,NSE,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
4904,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,None,0.0,0.05,1,EQ,NSE,NSE
4905,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,None,0.0,0.05,1,EQ,NSE,NSE
4906,979713,3827,ZUARIIND,ZUARI INDUSTRIES,0.0,None,0.0,0.05,1,EQ,NSE,NSE
4907,2029825,7929,ZYDUSLIFE,ZYDUS LIFESCIENCES,0.0,None,0.0,0.05,1,EQ,NSE,NSE


In [16]:
data = get_diffrent_time_data(instrument_token,start_dt, end_dt,interval)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
trades_list = [] 
for r,c in inst_filter.iterrows():
    print(c.tradingsymbol)
    instrument_token = c.instrument_token
    start_dt = datetime.datetime(2020,1,1)    # From last & days
    end_dt = datetime.datetime.now()
    interval = "day"
    try:
        data = get_diffrent_time_data(instrument_token,start_dt, end_dt,interval)
        #     data.date = pd.to_datetime(data.date,unit = "ms")
        data.date = pd.to_datetime(pd.to_datetime(data.date).dt.date)
        data.rename(columns = {'date': "Date"},inplace = True)
        data.set_index('Date', inplace = True)
        data.columns = ['Open','High','Low',"Close","Volumne"]

        applyindicator(data)
        conditions(data)
        trades = matchtrades(data)
        trades_list.append(trades)
    except:
        continue

20MICRONS
21STCENMGM
3IINFOLTD
3MINDIA
3PLAND
4THDIM
5PAISA
63MOONS
A2ZINFRA
AAKASH
AAREYDRUGS
AARON-BE
AARTIDRUGS
AARTIIND
AARTISURF
AARVEEDEN
AARVI
AAVAS
ABAN
ABB
ABBOTINDIA
ABCAPITAL
ABFRL
ABMINTLLTD-BE
ABSLAMC
ABSLBAINAV
ABSLBANETF
ABSLNN50ET
ABSLNNINAV
ACC
ACCELYA
ACCURACY
ACE
ACRYSIL
ADANIENT
ADANIGREEN
ADANIPORTS
ADANIPOWER-BE
ADANITRANS
ADFFOODS
ADL-BE
ADORWELD
ADROITINFO
ADSL
ADVANIHOTR
ADVENZYMES
AEGISCHEM
AETHER
AFFLE
AGARIND
AGI
AGRITECH
AGROPHOS
AGSTRA
AHLADA-BE
AHLEAST-BE
AHLUCONT
AIAENG
AIRAN
AIROLAM
AJANTPHARM
AJMERA
AJOONI
AJRINFRA
AKASH
AKG-BE
AKSHAR
AKSHARCHEM
AKSHOPTFBR
AKZOINDIA
ALANKIT
ALBERTDAVD
ALEMBICLTD
ALICON
ALKALI
ALKEM
ALKYLAMINE
ALLCARGO
ALLSEC
ALMONDZ
ALOKINDS-BE
ALPA
ALPHAGEO
ALPSINDUS
AMARAJABAT
AMBER
AMBICAAGAR-BE
AMBIKCO
AMBUJACEM
AMDIND-BE
AMIORG
AMJLAND
AMRUTANJAN
ANANDRATHI
ANANTRAJ
ANDHRACEMT-BE
ANDHRAPAP
ANDHRSUGAR
ANDREWYU
ANGELONE
ANIKINDS
ANKITMETAL
ANMOL
ANSALAPI-BE
ANTGRAPHIC
ANUP
ANURAS
APARINDS
APCL
APCOTEXIND
APEX
APLAPOLLO
APLLTD
APOLLO

In [ ]:
trade_df = pd.concat(trades_list)

In [60]:
trade_df[trade_df.index == '2022-10-17']

,Open,High,Low,Close,Volumne,sma200,sma20,stddev,Upper,Lower,rsi,Buy,Sell,Buyprice,Sellprice,profit
Date,,,,,,,,,,,,,,,,
2022-10-17,980.0,981.45,906.1,935.4,313321,895.106,989.7275,14.536257,1026.068142,953.386858,3.326881,1,0,907.338,939.4,NaN


In [55]:
trade_df['profit'] = (trade_df.Sellprice.shift(-1) - trade_df.Buyprice)/ trade_df.Buyprice  

In [57]:
trade_df.sort_values("profit")

,Open,High,Low,Close,Volumne,sma200,sma20,stddev,Upper,Lower,rsi,Buy,Sell,Buyprice,Sellprice,profit
Date,,,,,,,,,,,,,,,,
2021-03-23,83999.90,84999.95,83600.00,84615.10,26061,71719.56875,87208.7275,2090.865121,92435.890304,81981.564696,59.655453,0,1,82076.6470,83870.00,-0.999612
2022-05-13,41570.00,42989.90,41388.60,41787.60,25039,38836.70525,44525.7125,1816.791900,49067.692249,39983.732751,57.356847,0,1,40533.9720,41815.20,-0.998253
2022-01-31,3843.95,4070.00,3835.00,4006.30,1688744,3530.96700,4353.6000,379.013024,5301.132560,3406.067440,79.900773,0,1,3886.1110,4113.90,-0.996861
2022-01-28,4601.50,4825.00,4560.00,4713.25,1344133,4722.18800,5491.2650,480.745587,6693.128966,4289.401034,54.560672,0,1,4571.8525,4807.00,-0.996642
2021-10-27,10500.00,11516.65,10500.00,11095.80,13290,11644.45025,11806.9425,719.265732,13605.106830,10008.778170,78.889035,0,1,10762.9260,11123.90,-0.996576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-06,1.30,1.30,1.30,1.30,100,1.32725,1.4900,0.155259,1.878147,1.101853,80.062815,0,1,1.2610,1.30,838.333862
2021-08-12,3.40,3.60,3.40,3.55,136164,2.93975,3.6975,0.153447,4.081118,3.313882,69.274225,0,1,3.4435,3.65,1135.924641
2021-08-24,13.85,14.85,13.45,14.85,113757,11.88500,16.1275,0.782451,18.083629,14.171371,53.466690,0,1,14.4045,15.30,2939.768510
